In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Writing models using JAX's `stax` module

In this notebook, I'll show the code for how to use JAX's `stax` submodule to write arbitrary models.

## What is `stax`?

Most deep learning libraries use objects as the data structure for a neural network layer.
As such, the tunable parameters of the layer, for example `w` and `b` for a linear ("dense") layer
are class attributes associated with the forward function.

In some sense, because a neural network layer is nothing more than a math function,
specifying the layer in terms of a function might also make sense.
`stax`, then, is a new take on writing neural network models using pure functions rather than objects.

## How does `stax` work?

The way that `stax` layers work is as follows.
Every neural network layer is nothing more than a math function with a "forward" pass.
Neural network models typically have their parameters 
initialized into the right shapes using random number generators.
Put these two together, and we have a _pair_ of functions that specify a layer:

- An `init_fun` function, that _initializes_ parameters into the correct shapes, and
- An `apply_fun` function, that _applies_ the specified math transformations onto incoming data, using parameters of the correct shape.

## Example: Linear layer

Let's see an example of this in action, by studying the implementation of the linear ("dense") layer in `stax`

In [ ]:
from jax.experimental import stax

In [ ]:
stax.Dense??

As you can see, the `apply_fun` specifies the linear transformation.
It accepts a parameter called `params`,
which gets tuple-unpacked into the appropriate `W` and `b`.

**Notice how the `params` argument matches up with the second output of `init_fun`!**
The `init_fun` always accepts an `rng` parameter, which is returned from JAX's `jax.random.PRNGKey()`.
It also accepts an `input_shape` parameter,
which specifies what the elementary shape of one sample of data is.
So if your entire dataset is of shape `(n_samples, n_columns)`,
then you would put in `(n_columns,)` inside there,
as you would want to ignore the sample dimension.
(More on why this is the case later!)
The `init_fun` also returns the `output_shape`,
which is used later when we chain layers together.
(Trust me, it'll be fun.)

Let's see how we can use the Dense layer to specify a linear regression model.

### Create the initialization and application function pairs

Firstly, we create the `init_fun` and `apply_fun` pair:

In [ ]:
init_fun, apply_fun = stax.Dense(1)

### Initialize the parameters

Now, let's initialize parameters using the `init_fun`.

Let's assume that we have data that is of 1 column only, to keep things simple.

In [ ]:
from jax.random import PRNGKey, split, normal

key = PRNGKey(42)

output_shape, params = init_fun(key, input_shape=(1,))

In [ ]:
params

### Apply parameters and data through function

We'll create some randomly generated data.

In [ ]:
k1, k2 = split(key)
X = normal(k1, shape=(10, 1))
X

Here's some `y_true` values that I've snuck in.

In [ ]:
y_true = np.dot(X, 3) + 5
y_true

Now, we'll pass data through the linear model!

In [ ]:
apply_fun??

In [ ]:
from jax import vmap
from functools import partial

y_pred = vmap(partial(apply_fun, params))(data)
y_pred

Voilà! We have a simple linear model implemented just like that.

### What was that `vmap`???

`vmap` is a special function that JAX provides.
If we write all of our functions as behaving like they take in only one i.i.d. sample,
then we can use `vmap` to map our function over the inputs that are provided!

## Optimization

Next question: how do we *optimize* the parameters using JAX?

Instead of writing a training loop on our own, we can take advantage of JAX's optimizers, which are also written in a functional paradigm!

JAX's optimizers are constructed as a "triplet" set of functions:

- `init`: Takes `params` and initializes them in as a `state`, which is structured in a fashion that `update` can operate on.
- `update`: Takes in `i`, `g`, and `state`, which respectively are:
    - `i`: The current loop iteration
    - `g`: Gradients calculated from `grad`!
    - `state`: The current state of the parameters.
- `get_params`: Takes in the `state` at a given point, and returns the parameters structured correctly.

In [ ]:
from jax import jit
init, update, get_params = adam(step_size=1e-1)
update = jit(update)
get_params = jit(get_params)

### Loss Function

We're still missing a piece here, that is the loss function.
For illustration purposes, let's use the mean squared error.

In [ ]:
from jax import grad
import jax.numpy as np

def mseloss(params, model, x, y_true):
    y_preds = vmap(partial(model, params))(x)
    return np.mean(np.power(y_preds - y_true, 2))

dmseloss = grad(mseloss)

### "Step" portion of update loop

Now, we're going to define the "step" portion of the update loop.

In [ ]:
def step(i, state, dlossfunc, model, x, y_true):
    params = get_params(state)
    g = dlossfunc(params, model, x, y_true)
    state = update(i, g, state)
    return state

### JIT compilation

Because it takes so many parameters (in order to remain pure, and not rely on notebook state),
we're going to bind some of them using `functools.partial`.

I'm also going to show you what happens when we JIT-compile vs. don't JIT-compile the function.

In [ ]:
step_partial = partial(step, dlossfunc=dmseloss, model=apply_fun, x=X, y_true=y_true)
step_partial_jit = jit(step_partial)

### Explicit loops

Firstly, let's see what kind of code we'd write if we _did_ write the loop explicitly.

In [ ]:
from time import time
start = time()
state = init(params)
for i in range(1000):
    params = get_params(state)
    g = dmseloss(params, apply_fun, X, y_true)
    state = update(i, g, state)
end = time()
print(end - start)

### Partialled out loop step

Now, let's run the loop with the partialled out function.

In [ ]:
start = time()
state = init(params)
for i in range(1000):
    state = step_partial(i, state)
end = time()
print(end - start)


### JIT-compiled loop!

This is much cleaner of a loop, but we did have to do some work up-front.

What happens if we now use the JIT-ed function?

In [ ]:
start = time()
state = init(params)
for i in range(1000):
    state = step_partial_jit(i, state)
end = time()
print(end - start)



Whoa, holy smokes, that's fast! At least 10X faster using JIT-compilation.

In [ ]:
get_params(state)

Aaaaand..... we were able to recover the correct parameters!